In [17]:
import sys
sys.path.append('..')

In [18]:
import pandas as pd
from tqdm import tqdm
import time

from summarizers.light import SumyTextRankSummarizer, FastSummarizer 

In [19]:
df = pd.read_csv('../dataset/unified_dataset_labeled.csv')
print(df.shape)

(198, 10)


In [22]:
random_summarizer = SumyTextRankSummarizer('random')
lsa_summarizer = SumyTextRankSummarizer('lsa')
textrank_summarizer = SumyTextRankSummarizer('text_rank')
fast_summarizer = FastSummarizer()


summarizers = {
    'random': random_summarizer,
    'lsa': lsa_summarizer,
    'text_rank': textrank_summarizer,
    'fast': fast_summarizer
}

# kwargs to pass to the for each summarizer
params = {
}


sumy v2
sumy v2
sumy v2
fast summarizer v3


In [23]:
# Initialize columns
for name in summarizers:
    df[f"summary_{name}"] = None

# Generate
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Running summarizers"):
    text, lang = row["cleaned_markdown"], row["language"]
    for name, summarizer in summarizers.items():
        kw = params.get(name, {})
        
        # calculate latency
        start = time.perf_counter()
        summary = summarizer.summarize(text, lang, **kw)
        latency = time.perf_counter() - start
        
        df.at[idx, f"summary_{name}"] = summary
        df.at[idx, f"summary_{name}_latency"] = latency

Running summarizers: 100%|██████████| 198/198 [01:03<00:00,  3.13it/s]


In [24]:
# Save
df.to_csv("../dataset/results.csv", index=False)

In [25]:
# show latency statistics
for name in summarizers:
    latencies = df[f"summary_{name}_latency"].dropna()
    # convert to ms
    latencies = latencies * 1000  # convert to milliseconds
    print(f"{name} - Mean Latency: {latencies.mean():.4f}ms, Std: {latencies.std():.4f}ms, Min: {latencies.min():.4f}ms, Max: {latencies.max():.4f}ms")

random - Mean Latency: 8.2367ms, Std: 45.6223ms, Min: 1.0622ms, Max: 604.6899ms
lsa - Mean Latency: 44.0192ms, Std: 56.0383ms, Min: 1.2434ms, Max: 536.9721ms
text_rank - Mean Latency: 31.0331ms, Std: 49.5466ms, Min: 1.2042ms, Max: 546.4159ms
fast - Mean Latency: 232.0790ms, Std: 208.0558ms, Min: 0.4636ms, Max: 1583.8066ms
